In [ ]:
!pip install -q tensorflow-model-optimization

In [1]:
import tempfile
import os

import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Load data
student_df = pd.read_csv('../../datasets/student_mat_processed01.csv')

train_dataset = student_df.sample(frac=0.8, random_state=0)
test_dataset = student_df.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_G1_targets = train_features.pop('G1')
train_G2_targets = train_features.pop('G2')
train_G3_targets = train_features.pop('G3')

test_G1_targets = test_features.pop('G1')
test_G2_targets = test_features.pop('G2')
test_G3_targets = test_features.pop('G3')

In [3]:
# Normalizer Layer
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

In [11]:
# Define the model architecture.
def build_and_compile_model():
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(45)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(1)
    ])

    # Train the model
    model.compile(loss='mean_absolute_error', metrics=[tf.keras.metrics.MeanAbsoluteError()],
                  optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

In [12]:
fc_model = build_and_compile_model()
fc_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 32)                1472      
_________________________________________________________________
dense_9 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 2,145
Trainable params: 2,145
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = fc_model.fit(
    train_features, train_G3_targets,
    validation_split=0.2,
    verbose=1, epochs=100)

Epoch 1/100
8/8 [==============================] - 0s 41ms/step - loss: 9.2220 - mean_absolute_error: 9.2220 - val_loss: 9.3482 - val_mean_absolute_error: 9.3482
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 7.7513 - mean_absolute_error: 7.7513 - val_loss: 7.5890 - val_mean_absolute_error: 7.5890
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 6.2526 - mean_absolute_error: 6.2526 - val_loss: 5.9949 - val_mean_absolute_error: 5.9949
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 5.1715 - mean_absolute_error: 5.1715 - val_loss: 4.7786 - val_mean_absolute_error: 4.7786
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 4.4839 - mean_absolute_error: 4.4839 - val_loss: 3.9029 - val_mean_absolute_error: 3.9029
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: 4.0923 - mean_absolute_error: 4.0923 - val_loss: 3.5588 - val_mean_absolute_error: 3.5588
Epoch 7/100
8/8 [================

8/8 [==============================] - 0s 4ms/step - loss: 2.9106 - mean_absolute_error: 2.9106 - val_loss: 3.0940 - val_mean_absolute_error: 3.0940
Epoch 52/100
8/8 [==============================] - 0s 3ms/step - loss: 2.8744 - mean_absolute_error: 2.8744 - val_loss: 3.1222 - val_mean_absolute_error: 3.1222
Epoch 53/100
8/8 [==============================] - 0s 3ms/step - loss: 2.8727 - mean_absolute_error: 2.8727 - val_loss: 3.1259 - val_mean_absolute_error: 3.1259
Epoch 54/100
8/8 [==============================] - 0s 3ms/step - loss: 2.8447 - mean_absolute_error: 2.8447 - val_loss: 3.0458 - val_mean_absolute_error: 3.0458
Epoch 55/100
8/8 [==============================] - 0s 3ms/step - loss: 2.8349 - mean_absolute_error: 2.8349 - val_loss: 3.1718 - val_mean_absolute_error: 3.1718
Epoch 56/100
8/8 [==============================] - 0s 3ms/step - loss: 2.8454 - mean_absolute_error: 2.8454 - val_loss: 3.0416 - val_mean_absolute_error: 3.0416
Epoch 57/100
8/8 [=======================

In [14]:
test_results = {}
_, test_results['fc_model'] = fc_model.evaluate(test_features, test_G3_targets, verbose=0)

print('Baseline mean absolute error:', test_results['fc_model'])

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(fc_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline mean absolute error: 3.3839049339294434
Saved baseline model to: C:\Users\z0042fkb\AppData\Local\Temp\tmpt7_s684m.h5


In [15]:
import tensorflow_model_optimization as tfmot

In [16]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set.

num_features = train_features.shape[0] * (1 - validation_split)
end_step = np.ceil(num_features / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(fc_model, **pruning_params)

Instructions for updating:
Please use `layer.add_weight` method instead.


In [17]:
# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mean_absolute_error',
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

model_for_pruning.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_dense_8  (None, 32)                2914      
_________________________________________________________________
prune_low_magnitude_dense_9  (None, 16)                1042      
_________________________________________________________________
prune_low_magnitude_dense_10 (None, 8)                 266       
_________________________________________________________________
prune_low_magnitude_dense_11 (None, 1)                 19        
Total params: 4,241
Trainable params: 2,145
Non-trainable params: 2,096
_________________________________________________________________


In [18]:
# Training and evaluate the model against baseline
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_features, train_G3_targets,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
1/9 [==>...........................] - ETA: 0s - loss: 4.4794 - mean_absolute_error: 4.4794WARNING:tensorflow:From C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
9/9 [==============================] - 1s 74ms/step - loss: 2.9520 - mean_absolute_error: 2.9520 - val_loss: 3.2266 - val_mean_absolute_error: 3.2266
Epoch 2/2
9/9 [==============================] - 0s 7ms/step - loss: 2.8601 - mean_absolute_error: 2.8601 - val_loss: 3.2534 - val_mean_absolute_error: 3.2534


In [20]:
_, test_results['pruned_model'] = model_for_pruning.evaluate(
   test_features, test_G3_targets, verbose=1)

print('Baseline mean absolute error:', test_results['fc_model']) 
print('Pruned mean absolute error:', test_results['pruned_model'])

3/3 [==============================] - 0s 0s/step - loss: 3.2778 - mean_absolute_error: 3.2778
Baseline mean absolute error: 3.3839049339294434
Pruned mean absolute error: 3.277818441390991
Saved baseline model to: C:\Users\z0042fkb\AppData\Local\Temp\tmpn6odn1lf.h5


In [21]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: C:\Users\z0042fkb\AppData\Local\Temp\tmphundugai.h5


In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\z0042fkb\AppData\Local\Temp\tmp3lfhfjqm\assets
Saved pruned TFLite model to: C:\Users\z0042fkb\AppData\Local\Temp\tmp4f__pefl.tflite


In [23]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [24]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 9866.00 bytes
Size of gzipped pruned Keras model: 6795.00 bytes
Size of gzipped pruned TFlite model: 6005.00 bytes
